# Try to use w2v(与前文完全分离，包括包导入）

In [1]:
import pandas as pd
import numpy as np
import jieba
from jieba import analyse
import sklearn
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time

In [2]:
##################### 继续运行，读取数据
from gensim.models import Word2Vec
# isword = True
# model = Word2Vec.load("Douban_300features_40minwords_10context")
# review_vec_nda = np.load('review_vec_nda.npy')

isword = False
model = Word2Vec.load("Douban_300features_40minwords_10context_zi")
review_vec_nda = np.load('review_vec_nda_zi.npy')

C:\Users\czf10\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
train = pd.read_csv('DMSC.csv')

In [3]:
def get_word_list(sentens, isword=True):
    # 选择分词或分字
    if isword:
        return jieba.lcut(sentens)
    else:
        return list(sentens.strip())
# reviews = train[train['Movie_Name_CN'] == '钢铁侠1'].Comment.map(lambda x: jieba.lcut(x))
reviews = train.Comment.apply(get_word_list, args=(isword,))

In [4]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import Word2Vec
print("Training model...")
model = Word2Vec(reviews, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

C:\Users\czf10\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Training model...


In [5]:
model.init_sims(replace=True)

# model_name = "Douban_300features_40minwords_10context"
model_name = "Douban_300features_40minwords_10context_zi"
model.save(model_name)

In [4]:
# 将词汇表转换为set，加速
index2word_set = set(model.wv.index2word)

def get_vec(comment, isword=True):
    words = []
    if isword:
        comment2word = jieba.cut(comment)
    else:
        comment2word = list(comment.strip())
    for word in comment2word:
        if word in index2word_set:
            words.append(word)
    if not words: return np.zeros(model.wv.vector_size)
    return model.wv[words].mean(axis=0)

In [5]:
%%time
# 利用Series.map，比较快，生成Series
start = time.time()
print("Start time:{}".format(time.ctime(start)))

review_vec = train.Comment.apply(get_vec, args=(isword,))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\czf10\AppData\Local\Temp\jieba.cache


Start time:Mon Aug  6 14:12:03 2018


Loading model cost 0.761 seconds.
Prefix dict has been built succesfully.


Wall time: 11min 30s


In [6]:
# 将Series转换成ndarray
review_vec_nda = np.zeros((review_vec.shape[0], model.vector_size))

for i in range(review_vec.shape[0]):
    review_vec_nda[i] = review_vec[i]

In [15]:
# np.save('review_vec_nda', review_vec_nda)
# np.save('review_vec_nda_zi', review_vec_nda)

# Random Forest

In [14]:
#################### Random Forest

start = time.time()
print("Start time:", time.ctime(start))

parameters = {
     'n_estimators': (100, 200, 400),
     'max_depth': (16, 24),
     'min_samples_split': (8, 16),
     'min_samples_leaf': (4, 12, 16)     
#     'n_estimators': (200,),
#      'max_depth': (16,),
#      'min_samples_split': (12,),
#      'min_samples_leaf': (8,)
}
forest = RandomForestClassifier(oob_score=True, random_state=1, verbose=2)
clf = GridSearchCV(forest, parameters, cv=4, n_jobs=-1, verbose=5)

# choose_smp = np.random.choice(train.shape[0], 500000, replace=True)
choose_smp = train['Movie_Name_CN'] == '疯狂动物城'

clf.fit(review_vec_nda[choose_smp], train.Star[choose_smp])
print(clf.best_score_, clf.best_params_)

print("End time:", time.ctime(time.time()))
print('Use {} mins'.format((time.time()-start)/60))

Start time: Mon Aug  6 12:31:29 2018
Fitting 5 folds for each of 36 candidates, totalling 180 fits


JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\czf10\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\czf10\Anaconda3\lib\runpy.

In [8]:
##################### Random Forest(Best parameters)

%%time
start = time.time()
print("Start time:", time.ctime(start))

parameters = { 
    'n_estimators': (200,),
     'max_depth': (16,),
     'min_samples_split': (12,),
     'min_samples_leaf': (8,)
}
forest = RandomForestClassifier(oob_score=True, random_state=1, verbose=2)
clf = GridSearchCV(forest, parameters, cv=4, n_jobs=-1, verbose=5)

choose_smp = np.random.choice(train.shape[0], 600000, replace=True)
# choose_smp = train['Movie_Name_CN'] == '疯狂动物城'

# clf.fit(review_vec_nda, train.Star)
clf.fit(review_vec_nda[choose_smp], train.Star[choose_smp])
print(clf.best_score_, clf.best_params_)

print("End time:", time.ctime(time.time()))
print('Use {} mins'.format((time.time()-start)/60))

Start time: Mon Aug  6 19:51:49 2018
Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed: 60.9min remaining: 60.9min
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 62.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 62.1min finished


building tree 1 of 200


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.4s remaining:    0.0s


building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 93.9min finished


0.48064 {'max_depth': 16, 'min_samples_leaf': 8, 'min_samples_split': 12, 'n_estimators': 200}
End time: Mon Aug  6 22:31:58 2018
Use 160.15835042397183 mins
Wall time: 2h 40min 9s


# Kmeans

In [10]:
######################## Kmeans
%%time
start = time.time()
print("Start time:", time.ctime(start))

parameters = { 
    'n_clusters': (5,),
}
from sklearn.cluster import KMeans
kmeans = KMeans( verbose=2)
clf = GridSearchCV(kmeans, parameters, cv=4, n_jobs=-1, verbose=5)

# choose_smp = np.random.choice(train.shape[0], 600000, replace=True)
choose_smp = train['Movie_Name_CN'] == '疯狂动物城'

# clf.fit(review_vec_nda, train.Star)
clf.fit(review_vec_nda[choose_smp], train.Star[choose_smp])
print(clf.best_score_, clf.best_params_)

print("End time:", time.ctime(time.time()))

Start time: Wed Aug  8 12:07:05 2018
Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed: 10.5min remaining: 10.5min
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 11.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 11.9min finished


Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 12566.691919
start iteration
done sorting
end inner loop
Iteration 1, inertia 12175.9065452
start iteration
done sorting
end inner loop
Iteration 2, inertia 12053.2027475
start iteration
done sorting
end inner loop
Iteration 3, inertia 12014.5814289
start iteration
done sorting
end inner loop
Iteration 4, inertia 11998.6031702
start iteration
done sorting
end inner loop
Iteration 5, inertia 11990.191344
start iteration
done sorting
end inner loop
Iteration 6, inertia 11986.1333626
start iteration
done sorting
end inner loop
Iteration 7, inertia 11981.3658421
start iteration
done sorting
end inner loop
Iteration 8, inertia 11978.9371515
start iteration
done sorting
end inner loop
Iteration 9, inertia 11978.0143654
start iteration
done sorting
end inner loop
Iteration 10, inertia 11977.6217901
start iteration
done sorting
end inner loop
Iteration 11, inertia 11977.4617919
start iteration
done sorting

Iteration 24, inertia 12036.9142533
start iteration
done sorting
end inner loop
Iteration 25, inertia 12036.9131935
start iteration
done sorting
end inner loop
Iteration 26, inertia 12036.9128138
center shift 1.762380e-04 within tolerance 3.674804e-08
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 12613.4801626
start iteration
done sorting
end inner loop
Iteration 1, inertia 12323.4889658
start iteration
done sorting
end inner loop
Iteration 2, inertia 12194.2701028
start iteration
done sorting
end inner loop
Iteration 3, inertia 12141.01291
start iteration
done sorting
end inner loop
Iteration 4, inertia 12114.0419866
start iteration
done sorting
end inner loop
Iteration 5, inertia 12094.4451238
start iteration
done sorting
end inner loop
Iteration 6, inertia 12078.9498459
start iteration
done sorting
end inner loop
Iteration 7, inertia 12067.6576156
start iteration
done sorting
end inner loop
Iteration 8, inertia 12059.7985028
start iteration

Iteration 11, inertia 12163.2256907
start iteration
done sorting
end inner loop
Iteration 12, inertia 12161.5166594
start iteration
done sorting
end inner loop
Iteration 13, inertia 12160.378568
start iteration
done sorting
end inner loop
Iteration 14, inertia 12159.5836142
start iteration
done sorting
end inner loop
Iteration 15, inertia 12159.0807013
start iteration
done sorting
end inner loop
Iteration 16, inertia 12158.7072384
start iteration
done sorting
end inner loop
Iteration 17, inertia 12158.462339
start iteration
done sorting
end inner loop
Iteration 18, inertia 12158.1416721
start iteration
done sorting
end inner loop
Iteration 19, inertia 12157.9580969
start iteration
done sorting
end inner loop
Iteration 20, inertia 12157.8420319
start iteration
done sorting
end inner loop
Iteration 21, inertia 12157.7599582
start iteration
done sorting
end inner loop
Iteration 22, inertia 12157.7028459
start iteration
done sorting
end inner loop
Iteration 23, inertia 12157.6588013
start 

Iteration 2, inertia 12829.5124351
start iteration
done sorting
end inner loop
Iteration 3, inertia 12693.0775392
start iteration
done sorting
end inner loop
Iteration 4, inertia 12627.3193191
start iteration
done sorting
end inner loop
Iteration 5, inertia 12569.6197621
start iteration
done sorting
end inner loop
Iteration 6, inertia 12499.9371551
start iteration
done sorting
end inner loop
Iteration 7, inertia 12422.5201331
start iteration
done sorting
end inner loop
Iteration 8, inertia 12363.9953391
start iteration
done sorting
end inner loop
Iteration 9, inertia 12314.1839166
start iteration
done sorting
end inner loop
Iteration 10, inertia 12285.3408072
start iteration
done sorting
end inner loop
Iteration 11, inertia 12267.053939
start iteration
done sorting
end inner loop
Iteration 12, inertia 12237.6139972
start iteration
done sorting
end inner loop
Iteration 13, inertia 12219.3243099
start iteration
done sorting
end inner loop
Iteration 14, inertia 12205.4031736
start iterati

Iteration 23, inertia 12029.4282971
start iteration
done sorting
end inner loop
Iteration 24, inertia 12024.9954989
start iteration
done sorting
end inner loop
Iteration 25, inertia 12021.2211655
start iteration
done sorting
end inner loop
Iteration 26, inertia 12014.3900705
start iteration
done sorting
end inner loop
Iteration 27, inertia 12007.018949
start iteration
done sorting
end inner loop
Iteration 28, inertia 12002.9135299
start iteration
done sorting
end inner loop
Iteration 29, inertia 12000.570283
start iteration
done sorting
end inner loop
Iteration 30, inertia 11999.069232
start iteration
done sorting
end inner loop
Iteration 31, inertia 11997.9704664
start iteration
done sorting
end inner loop
Iteration 32, inertia 11997.2792051
start iteration
done sorting
end inner loop
Iteration 33, inertia 11996.7928476
start iteration
done sorting
end inner loop
Iteration 34, inertia 11996.4585185
start iteration
done sorting
end inner loop
Iteration 35, inertia 11996.2667017
start i

In [25]:
clf.score(review_vec_nda, train.Star)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  2.5min finished


0.32555377364172994

# Other test

In [202]:
forest = RandomForestClassifier(n_estimators=200, max_depth=16, min_impurity_split=12, min_samples_leaf=8, oob_score=True, random_state=1, verbose=2)


(2125056, 300)
ID                                    0
Movie_Name_EN    Avengers Age of Ultron
Movie_Name_CN                    复仇者联盟2
Crawl_Date                   2017-01-22
Number                                1
Username                             然潘
Date                         2015-05-13
Star                                  3
Comment                   连奥创都知道整容要去韩国。
Like                               2404
Name: 0, dtype: object


In [15]:
%who

GridSearchCV	 RandomForestClassifier	 Word2Vec	 analyse	 choose_smp	 clf	 forest	 jieba	 model	 
np	 parameters	 pd	 review_vec_nda	 sklearn	 start	 time	 train	 


In [9]:
review_vec.size

2125056

In [3]:
?svm.SVC